# **Transformers**


## **1. DistilBERT**


### **Bias**

En este experimento se aborda la clasificación automática del sesgo en noticias utilizando técnicas de Deep Learning basadas en Transformers, específicamente DistilBERT. El objetivo es construir un modelo capaz de predecir si un texto pertenece a una de tres categorías de sesgo, aprendiendo a capturar patrones semánticos y estilísticos del lenguaje sin intervención manual en la extracción de características.

DistilBERT es una versión ligera de BERT, preentrenada sobre grandes corpus de texto en inglés. Su principal ventaja es un menor tamaño y menor número de parametros conservando un 97% de la precision de BERT 

Los objetivos que esperamos son los siguientes:

- Entrenar un modelo DistilBERT fine-tuneado sobre un subset del dataset de noticias
- Evaluar el rendimiento en términos de accuracy y macro F1, métricas apropiadas para clasificación multiclase, especialmente con clases desbalanceadas.
- Identificar si el modelo es capaz de captar patrones semánticos que diferencian los distintos tipos de sesgo en el contenido textual.

También esperamos varias complicaciones siendo la primera la falta de potencia computacional por nuestra parte lo que no puede llevar a problemas como truncamiento de texto,tamaño de dataset y clases desbalanceadas al intentar limitar el modelo por la falta de potencia que tenemos 

Nuestras expectativas son conseguir que el modelo capture relaciones semanticas generales aunque no sean perfectas ni mucho menos y conseguir que sea un modelo lo suficientemente bueno par aque se pudiese usar como base para mejoras futuras con mejores capacidades y el uso de GPU

#### 1. Carga y Preparación de Datos

Para entrenar un modelo de NLP como DistilBERT es fundamental partir de un dataset limpio y bien estructurado. En este caso se utiliza train_clean.csv, un archivo previamente procesado que contiene noticias ya depuradas. Este dataset ha pasado por pasos de limpieza de texto, tales como:

- Eliminación de caracteres especiales, enlaces y signos de puntuación innecesarios.
- Normalización de mayúsculas/minúsculas.
- Corrección de espacios y saltos de línea.

Del dataset completo se seleccionan solo las columnas necesarias que son content_clean (el texto de la noticia ya procesado y listo para tokenizar) y  bias (la etiqueta que indica el tipo de sesgo de la noticia, que puede tener tres posibles valores). Esta selección garantiza que el modelo reciba únicamente la información relevante y evita ruido innecesario que podría afectar el aprendizaje.

Una vez filtradas las columnas, se procede a dividir el dataset en tres subconjuntos:

- Entrenamiento (Train): aproximadamente el 72% del dataset total. Se utiliza para ajustar los pesos del modelo durante el fine-tuning y es importante que contenga ejemplos de todas las clases para que el modelo pueda aprender las diferencias semánticas entre ellas.

- Validación (Validation): alrededor del 8% del dataset total. Se utiliza durante el entrenamiento para evaluar el modelo en datos no vistos y ajustar hiperparámetros, como el learning rate o el número de epochs y nos ayuda a detectar overfitting, ya que si el modelo mejora en el train pero empeora en validación, significa que no generaliza bien.

- Prueba (Test): 20% restante del dataset. Este conjunto solo se utiliza después de finalizar el entrenamiento para medir el rendimiento real del modelo en datos completamente nuevos y nos permite calcular métricas como accuracy, macro F1, precision y recall de manera objetiva.

Para asegurar que cada conjunto contenga una proporción representativa de cada clase, se usa el parámetro stratify de train_test_split, esto evita que algunas clases queden subrepresentadas en el conjunto de validación o test, lo que podría sesgar la evaluación y dar métricas poco fiables y es especialmente importante en datasets desbalanceados, donde algunas clases podrían ser minoritarias

In [1]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)


c:\Users\SISTERS\miniconda3\envs\nlp310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Cargamos el dataset
df = pd.read_csv("data/data_clean/train_clean.csv")

# Nos quedamos solo con las columnas relevantes para la tarea
df = df[["content_clean", "bias"]].dropna()  # Eliminamos filas con valores faltantes

# Mostramos las primeras filas del DataFrame 
df.head()


,content_clean,bias
0,besides his most recent trip to quetta mr raha...,0
1,poll prestigious colleges wo nt make you happi...,0
2,house speaker paul ryan at a private dinner ea...,2
3,cnn president donald trump has reason to hope ...,0
4,the controversial immigrationreform bill that ...,2


In [ ]:
# Dividimos los datos originales en conjuntos de entrenamiento y prueba
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["content_clean"].tolist(),  # Lista de textos limpios
    df["bias"].tolist(),           # Lista de etiquetas de sesgo
    test_size=0.2,                 # 20% de los datos se reservan para el test
    random_state=42,               # Semilla para reproducibilidad
    stratify=df["bias"]             
)

# Dividimos el conjunto de entrenamiento en entrenamiento y validación
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts,                    
    train_labels,                   
    test_size=0.1,                  
    random_state=42,                
    stratify=train_labels            
)

# Mostramos la cantidad de ejemplos en cada conjunto
print(f"Train: {len(train_texts)}")
print(f"Validation: {len(val_texts)}")
print(f"Test: {len(test_texts)}")


Train: 20143
Validation: 2239
Test: 5596
Train: 20143
Validation: 2239
Test: 5596


####  2. Tokenization

Antes de alimentar los textos al modelo, es necesario convertirlos en una representación que DistilBERT pueda procesar: los tokens. Para ello, utilizamos el DistilBertTokenizerFast, que transforma cada noticia en una secuencia de IDs numéricos correspondientes a subpalabras reconocidas por el modelo.

Se aplica truncamiento y padding para que todas las secuencias tengan la misma longitud máxima (128 tokens en este caso). Esto permite que los lotes de entrenamiento sean consistentes y que el modelo pueda procesarlos de forma eficiente.

Se tokenizan conjuntos limitados de datos (train, validation y test) para reducir el tiempo de ejecución en CPU, seleccionando los primeros 3000, 1000 y 1000 ejemplos respectivamente. Esta estrategia permite experimentar de manera rápida sin perder la representatividad general de los datos, aunque sacrifica parte de la cobertura completa del dataset.

In [ ]:
# Cargamos el tokenizador rápido de DistilBERT preentrenado
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"  # Tokenizador compatible con el modelo preentrenado
)

# Función que tokeniza una lista de textos y los prepara para el modelo
def tokenize(texts):
    return tokenizer(
        texts,
        truncation=True,             # Trunca los textos que exceden la longitud máxima
        padding="max_length",        # Rellena los textos más cortos hasta la longitud máxima
        max_length=128,              # Longitud máxima de tokens por entrada
        return_tensors="pt"          # Devuelve los tensores 
    )

# Limitar la cantidad de datos 
max_train = 3000
max_val   = 1000
max_test  = 1000

train_encodings = tokenize(train_texts[:max_train])  # Tokenizamos el conjunto de entrenamiento
val_encodings   = tokenize(val_texts[:max_val])      # Tokenizamos el conjunto de validación
test_encodings  = tokenize(test_texts[:max_test])    # Tokenizamos el conjunto de test


####  3. Dataset

Una vez que los textos se han convertido en representaciones numéricas mediante la tokenización, el siguiente paso es organizarlos junto con sus etiquetas para que el modelo pueda aprender de manera eficiente. Aquí es donde entran en juego los Datasets de PyTorch.

Los Datasets actúan como contenedores estructurados de los datos, que permiten:

1. Almacenar y vincular datos y etiquetas de forma coherente:
Cada ejemplo del Dataset contiene tanto la representación numérica del texto como su etiqueta correspondiente. Esto asegura que cada batch extraído durante el entrenamiento tenga la información completa necesaria para que el modelo aprenda la relación entre entrada y salida.

2. Facilitar el acceso por índice:
El modelo y el Trainer necesitan poder acceder rápidamente a ejemplos individuales o batches completos. El Dataset permite extraer datos por índice, lo que habilita operaciones como el entrenamiento en lotes (mini-batches) y el submuestreo de los datos para pruebas rápidas o validaciones parciales.

3. Optimizar el manejo de memoria y procesamiento:
Los conjuntos de datos pueden ser muy grandes, y no siempre es posible cargarlos completamente en memoria. Al estructurar los datos como Dataset, es posible trabajar con subconjuntos y cargar solo lo necesario para cada batch 

4. Compatibilidad con herramientas de PyTorch y Transformers:
La creación de Datasets permite integrarse de manera fluida con los DataLoaders y el Trainer de Hugging Face. Esto automatiza la creación de batches, la gestión de tensores, el cálculo de métricas y la evaluación del modelo, haciendo el flujo de entrenamiento más eficiente y reproducible.

5. Flexibilidad para experimentación:
Al usar Datasets, se puede fácilmente crear subconjuntos de entrenamiento, validación o test, controlar la cantidad de ejemplos procesados, y ajustar rápidamente los experimentos sin modificar los datos originales.

In [5]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Para cada ejemplo devolvemos los tensores de input_ids, attention_mask y la etiqueta
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Crear datasets de entrenamiento, validación y test
train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset   = NewsDataset(val_encodings, val_labels)
test_dataset  = NewsDataset(test_encodings, test_labels)


train_subset = Subset(train_dataset, range(min(3000, len(train_dataset))))
val_subset   = Subset(val_dataset,   range(min(1000, len(val_dataset))))
test_subset  = Subset(test_dataset,  range(min(1000, len(test_dataset))))


####  4. Enternamiento

El entrenamiento del modelo DistilBERT consiste en ajustar los pesos del modelo preentrenado para que pueda clasificar correctamente los textos de noticias según su sesgo. Partimos de un modelo ya preentrenado, lo que significa que DistilBERT ya ha aprendido representaciones lingüísticas generales a partir de grandes corpus de texto. Durante esta fase, estas representaciones se refinan específicamente para nuestro problema de clasificación de sesgo. Esto permite que el modelo aprenda patrones semánticos y contextuales relevantes para distinguir entre las tres clases de sesgo definidas en nuestro dataset.

Para hacer el entrenamiento más manejable y eficiente, se trabaja sobre un subset limitado de los datos, seleccionando solo unos miles de ejemplos de entrenamiento y validación. Esto reduce el tiempo de entrenamiento, especialmente importante cuando se trabaja en CPU, y permite iterar más rápidamente sobre la configuración de los hiperparámetros.

El proceso de entrenamiento se guía por una función de pérdida que compara las predicciones del modelo con las etiquetas reales, ajustando los pesos para minimizar el error. Al mismo tiempo, se aplican técnicas como la regularización por weight decay y un pequeño calentamiento de los pasos iniciales (warmup) para estabilizar el aprendizaje y evitar que el modelo se desestabilice al principio.

Durante cada epoch, el modelo revisa todos los ejemplos de entrenamiento, actualizando gradualmente sus parámetros. Además, se registran métricas de desempeño como la pérdida, para monitorizar cómo mejora el modelo a lo largo del tiempo. Este entrenamiento permite que DistilBERT adapte sus embeddings y capas internas a nuestro conjunto de noticias, buscando extraer patrones discriminativos útiles para la clasificación de sesgo.

In [ ]:
# Cargamos el modelo DistilBERT preentrenado para clasificación de secuencias
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",  
    num_labels=3                 # Número de clases de salida para la tarea de clasificación
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred                # Recibe la salida del modelo y las etiquetas reales
    preds = np.argmax(logits, axis=1)        # Convierte los logits en la clase predicha
    
    acc = accuracy_score(labels, preds)      # Calcula Accuracy
    f1  = f1_score(labels, preds, average="macro")  # Calcula Macro-F1 
    
    return {
        "accuracy": acc,
        "macro_f1": f1
    }



In [ ]:
# Configuración para el trainer
training_args = TrainingArguments(
    output_dir="./results",                   
    num_train_epochs=3,                      # Num de epochs
    per_device_train_batch_size=8,           # Tamaño del batch durante el entrenamiento
    per_device_eval_batch_size=16,           # Tamaño del batch durante la evaluación
    warmup_steps=20,                         # Num de steps
    weight_decay=0.01,                       # weight decay
    logging_steps=100,                       # loggear
    save_steps=500,                           # guardar modelo
    save_total_limit=1,                       # Limita checkpoints 
    no_cuda=True,                             
    eval_strategy="epoch",                    # Evaluar el modelo al final de cada epoch
    report_to="none"                        
)


c:\Users\SISTERS\miniconda3\envs\nlp310\lib\site-packages\transformers\training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [ ]:


train_subset = Subset(train_dataset, range(3000))  
val_subset   = Subset(val_dataset, range(2500))    

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset, 
    eval_dataset=val_subset,    
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


C:\Users\SISTERS\AppData\Local\Temp\ipykernel_4552\3333358261.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Entrenamos el modelo usando el trainer configurado previamente
train_output = trainer.train()

#información  del entrenamiento
print("\n=== Train Output ===")
print(train_output)  


C:\Users\SISTERS\AppData\Local\Temp\ipykernel_4552\97867553.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.944100,0.885592,0.597000,0.588982
2,0.713500,0.865070,0.626000,0.622499
3,0.368800,1.075154,0.624000,0.621700


C:\Users\SISTERS\AppData\Local\Temp\ipykernel_4552\97867553.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\SISTERS\AppData\Local\Temp\ipykernel_4552\97867553.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\SISTERS\AppData\Local\Temp\ipykernel_4552\97867553.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in s


=== Train Output ===
TrainOutput(global_step=1125, training_loss=0.6955495376586914, metrics={'train_runtime': 5817.322, 'train_samples_per_second': 1.547, 'train_steps_per_second': 0.193, 'total_flos': 298056962304000.0, 'train_loss': 0.6955495376586914, 'epoch': 3.0})


####  5. Evaluación

Tras el entrenamiento de DistilBERT con nuestro subconjunto de datos, procedimos a evaluar el rendimiento del modelo tanto en el conjunto de validación como en el de test, utilizando métricas estándar de clasificación: accuracy, macro-F1, y pérdida (loss). Estas métricas permiten valorar no solo la proporción de aciertos totales, sino también cómo se comporta el modelo frente a clases menos representadas.

En las primeras epochs, los resultados de validación mostraban una mejora gradual del accuracy y macro-F1, mientras que la pérdida disminuía, lo que indica que el modelo estaba aprendiendo a discriminar entre las clases. Por ejemplo, en la primera epoch el accuracy rondaba el 59% y el macro-F1 estaba en torno a 0.59. Esto refleja que aunque el modelo estaba captando ciertas relaciones entre palabras y sesgos de los textos, aún no era capaz de generalizar completamente.

A medida que avanzábamos a la segunda y tercera epoch, observamos que el accuracy y macro-F1 aumentaban ligeramente hasta valores cercanos al 62-64%, mientras que la pérdida mostraba un comportamiento más fluctuante. Este aumento se debe a que el modelo va refinando los embeddings internos y ajustando los pesos, captando patrones más complejos de lenguaje que permiten diferenciar mejor entre noticias con sesgos distintos. La ligera subida en la pérdida en la tercera epoch se puede explicar por el tamaño reducido del subconjunto de validación y la limitación de ejemplos, lo que genera cierta variabilidad en la evaluación.

El informe de clasificación final sobre el conjunto de test confirma esta tendencia. Las métricas por clase muestran que el modelo tiene mejor desempeño en las clases más frecuentes o con patrones lingüísticos más claros, mientras que las clases más heterogéneas o menos representadas presentan valores ligeramente inferiores de recall y f1-score. Esto es consistente con los resultados de macro-F1, que pondera todas las clases de manera equitativa, mostrando que el modelo generaliza razonablemente pero todavía tiene margen de mejora.

Estos resultados se lograron gracias a varias decisiones clave en el proceso: la pretokenización y truncado de textos a una longitud máxima de 128 tokens permitió que el modelo procesara la información de forma eficiente, mientras que el uso de subconjuntos representativos para entrenamiento y validación redujo drásticamente los tiempos de ejecución, manteniendo patrones significativos en los datos. Además, la configuración de DistilBERT como modelo base, facilitó entrenar varias epochs sin necesidad de GPU, consiguiendo que los embeddings se ajustaran a nuestro problema específico de clasificación de sesgo en noticias.

En conjunto, los resultados indican que el modelo ha logrado capturar relaciones semánticas útiles para la clasificación de bias, aunque su desempeño sigue limitado por el tamaño del dataset y la reducción de ejemplos para acelerar el entrenamiento. Para mejorar, sería clave aumentar la cantidad de datos, permitir que el modelo vea más ejemplos por clase y considerar técnicas de fine-tuning más profundo o aumento de datos, lo que podría ayudar a que las clases menos representadas obtengan mayor f1-score y que la pérdida de validación se estabilice aún más.

In [ ]:
# Evaluamos el modelo usando los datos de validación
eval_results = trainer.evaluate()
print("\n=== Validation Results ===")
# Mostramos los resultados de validación
for k, v in eval_results.items():
    print(f"{k}: {v:.4f}")  # 4 decimales 

# Realizamos predicciones sobre el subconjunto de test
preds_output = trainer.predict(test_subset)
# Convertimos las predicciones del modelo a etiquetas finales tomando la clase con mayor probabilidad
preds = np.argmax(preds_output.predictions, axis=1)
# Obtenemos las etiquetas reales del conjunto de test para compararlas
labels = preds_output.label_ids

# Mostramos el informe de clasificación con métricas como precisión, recall y F1-score
print("\n=== Test Classification Report ===")
print(classification_report(labels, preds, digits=4))  

# Guardamos el modelo e
trainer.save_model("models/distilbert_debug")
# Guardamos el tokenizador 
tokenizer.save_pretrained("models/distilbert_debug")


C:\Users\SISTERS\AppData\Local\Temp\ipykernel_4552\97867553.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}



=== Validation Results ===
eval_loss: 1.0752
eval_accuracy: 0.6240
eval_macro_f1: 0.6217
eval_runtime: 121.7037
eval_samples_per_second: 8.2170
eval_steps_per_second: 0.5180
epoch: 3.0000


C:\Users\SISTERS\AppData\Local\Temp\ipykernel_4552\97867553.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}



=== Test Classification Report ===
              precision    recall  f1-score   support

           0     0.6254    0.6218    0.6236       349
           1     0.6546    0.6059    0.6293       269
           2     0.6609    0.6990    0.6794       382

    accuracy                         0.6470      1000
   macro avg     0.6470    0.6422    0.6441      1000
weighted avg     0.6468    0.6470    0.6464      1000



('models/distilbert_debug\\tokenizer_config.json',
 'models/distilbert_debug\\special_tokens_map.json',
 'models/distilbert_debug\\vocab.txt',
 'models/distilbert_debug\\added_tokens.json',
 'models/distilbert_debug\\tokenizer.json')

#### 6, Comparación con otros modelos

En la tarea de clasificación de bias, los modelos basados en representaciones tradicionales de texto continúan mostrando el mejor rendimiento. En particular, TF-IDF alcanza una Accuracy de 0.71 y un Macro-F1 de 0.70, lo que indica que la información basada en la frecuencia y relevancia de términos captura de manera muy efectiva los patrones discriminativos necesarios para esta tarea. Bag-of-Words, aunque ligeramente inferior, sigue mostrando un desempeño sólido, con una Accuracy de 0.63 y un Macro-F1 de 0.63, reflejando la importancia de considerar la presencia de palabras clave concretas en lugar de su contexto semántico más amplio.

Los embeddings no contextuales, como Word2Vec y FastText, obtienen resultados intermedios, con Accuracy alrededor de 0.54 y Macro-F1 de 0.53–0.54. Estos modelos capturan relaciones semánticas entre palabras, pero al aplicarlos con un clasificador lineal como Logistic Regression, no superan el rendimiento de TF-IDF. Esto sugiere que, para la clasificación de bias, la mera presencia de términos clave es más determinante que la información semántica distribuida que estos embeddings aportan.

Los embeddings contextuales, incluyendo Sentence Transformers y BERT, muestran un comportamiento mixto. Sentence Transformers alcanza una Accuracy de 0.535 y un Macro-F1 de 0.535, similar a los embeddings no contextuales. Por otro lado, BERT sin un entrenamiento exhaustivo obtiene un desempeño muy bajo (Accuracy 0.366, Macro-F1 0.366). Esto refleja la dificultad de ajustar modelos de gran capacidad con datasets limitados y clases desbalanceadas, donde el modelo puede sobreajustarse o no generalizar correctamente.

Al comparar estos resultados con DistilBERT fine-tuneado, se observa un patrón coherente. DistilBERT, aunque ya se ha ajustado a la tarea de bias, sigue sin superar a TF-IDF en Accuracy o Macro-F1. La explicación principal es que DistilBERT está diseñado para capturar relaciones contextuales complejas y matices semánticos, pero la clasificación de bias depende en gran medida de palabras y expresiones específicas. En este caso, la riqueza contextual de DistilBERT no se traduce automáticamente en mejor rendimiento, especialmente cuando el conjunto de datos es pequeño y el fine-tuning no es suficiente para extraer patrones fiables. Por el contrario, TF-IDF, al enfatizar la frecuencia relativa de términos clave, resulta más robusto y eficaz para esta tarea concreta.

En conclusión, aunque DistilBERT fine-tuneado aporta capacidades contextuales avanzadas, los modelos tradicionales como TF-IDF siguen siendo más efectivos para la clasificación de bias en datasets limitados. Los embeddings no contextuales y contextuales muestran un desempeño moderado, pero no logran superar a las técnicas basadas en frecuencia de términos debido a la naturaleza de la tarea y la dependencia de palabras clave específicas.

### **Topic and Source**

#### 1. Carga de datos

In [32]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

# Cargamos el dataset limpio desde CSV
df = pd.read_csv("data/data_clean/train_clean.csv")

# Nos quedamos solo con las columnas relevantes para la tarea (source y topic)
# Ejemplo: para clasificar la fuente
df_source = df[["content_clean", "source"]].dropna()  # Eliminamos filas con valores faltantes
# Ejemplo: para clasificar el tema
df_topic = df[["content_clean", "topic"]].dropna()    # Eliminamos filas con valores faltantes

# Mostramos las primeras filas para verificar la carga
df_source.head()
df_topic.head()

,content_clean,topic
0,besides his most recent trip to quetta mr raha...,terrorism
1,poll prestigious colleges wo nt make you happi...,education
2,house speaker paul ryan at a private dinner ea...,us_house
3,cnn president donald trump has reason to hope ...,white_house
4,the controversial immigrationreform bill that ...,immigration


In [45]:
# Filtrar clases con al menos 2 ejemplos para evitar errores con stratify
counts_source = df_source['source'].value_counts()
df_source_filtered = df_source[df_source['source'].isin(counts_source[counts_source >= 2].index)]

# Dividir en train/test
train_texts_source, test_texts_source, train_labels_source, test_labels_source = train_test_split(
    df_source_filtered["content_clean"].tolist(),
    df_source_filtered["source"].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df_source_filtered["source"]
)

# Dividir train/val
train_texts_source, val_texts_source, train_labels_source, val_labels_source = train_test_split(
    train_texts_source,
    train_labels_source,
    test_size=0.1,
    random_state=42,
    stratify=train_labels_source
)

# Crear diccionario para mapear etiquetas a enteros
source_classes = list(set(train_labels_source + val_labels_source + test_labels_source))
source2id = {cls: i for i, cls in enumerate(source_classes)}

# Mapear etiquetas
train_labels_source = [source2id[label] for label in train_labels_source]
val_labels_source   = [source2id[label] for label in val_labels_source]
test_labels_source  = [source2id[label] for label in test_labels_source]

print(f"Source - Train: {len(train_texts_source)}, Validation: {len(val_texts_source)}, Test: {len(test_texts_source)}")


# --- TOPIC ---

# Filtrar clases con al menos 2 ejemplos
counts_topic = df_topic['topic'].value_counts()
df_topic_filtered = df_topic[df_topic['topic'].isin(counts_topic[counts_topic >= 2].index)]

# Dividir en train/test
train_texts_topic, test_texts_topic, train_labels_topic, test_labels_topic = train_test_split(
    df_topic_filtered["content_clean"].tolist(),
    df_topic_filtered["topic"].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df_topic_filtered["topic"]
)

# Dividir train/val
train_texts_topic, val_texts_topic, train_labels_topic, val_labels_topic = train_test_split(
    train_texts_topic,
    train_labels_topic,
    test_size=0.1,
    random_state=42,
    stratify=train_labels_topic
)

# Crear diccionario para mapear etiquetas a enteros
topic_classes = list(set(train_labels_topic + val_labels_topic + test_labels_topic))
topic2id = {cls: i for i, cls in enumerate(topic_classes)}

# Mapear etiquetas
train_labels_topic = [topic2id[label] for label in train_labels_topic]
val_labels_topic   = [topic2id[label] for label in val_labels_topic]
test_labels_topic  = [topic2id[label] for label in test_labels_topic]

print(f"Topic - Train: {len(train_texts_topic)}, Validation: {len(val_texts_topic)}, Test: {len(test_texts_topic)}")



Source - Train: 20106, Validation: 2234, Test: 5586
Topic - Train: 20143, Validation: 2239, Test: 5596


#### 2. Tokenizer

In [46]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

max_train = 3000
max_val   = 1000
max_test  = 1000

# Source
train_encodings_source = tokenize(train_texts_source[:max_train])
val_encodings_source   = tokenize(val_texts_source[:max_val])
test_encodings_source  = tokenize(test_texts_source[:max_test])

# Topic
train_encodings_topic = tokenize(train_texts_topic[:max_train])
val_encodings_topic   = tokenize(val_texts_topic[:max_val])
test_encodings_topic  = tokenize(test_texts_topic[:max_test])



#### 3. Dataset

In [53]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Source
train_dataset_source = NewsDataset(train_encodings_source, train_labels_source)
val_dataset_source   = NewsDataset(val_encodings_source, val_labels_source)
test_dataset_source  = NewsDataset(test_encodings_source, test_labels_source)

# Topic
train_dataset_topic = NewsDataset(train_encodings_topic, train_labels_topic)
val_dataset_topic   = NewsDataset(val_encodings_topic, val_labels_topic)
test_dataset_topic  = NewsDataset(test_encodings_topic, test_labels_topic)


# --- SUBCONJUNTOS LIMITADOS ---

train_subset_source = Subset(train_dataset_source, range(min(2000, len(train_dataset_source))))
val_subset_source   = Subset(val_dataset_source, range(min(800, len(val_dataset_source))))
test_subset_source  = Subset(test_dataset_source, range(min(800, len(test_dataset_source))))

train_subset_topic = Subset(train_dataset_topic, range(min(2000, len(train_dataset_topic))))
val_subset_topic   = Subset(val_dataset_topic, range(min(800, len(val_dataset_topic))))
test_subset_topic  = Subset(test_dataset_topic, range(min(800, len(test_dataset_topic))))

#### 4. Entrenamiento

In [54]:
# Número de clases calculado dinámicamente
num_labels_source = len(df_source["source"].unique())
num_labels_topic  = len(df_topic["topic"].unique())

# --- Source ---
model_source = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels_source
)

# --- Topic ---
model_topic = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels_topic
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    acc = accuracy_score(labels, preds)
    f1  = f1_score(labels, preds, average="macro")
    
    return {"accuracy": acc, "macro_f1": f1}



In [56]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=20,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=500,
    save_total_limit=1,
    no_cuda=True,              # Forzar CPU
    eval_strategy="epoch",
    report_to="none"
)

In [57]:

# --- Source ---
trainer_source = Trainer(
    model=model_source,
    args=training_args,
    train_dataset=train_subset_source,
    eval_dataset=val_subset_source,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# --- Topic ---
trainer_topic = Trainer(
    model=model_topic,
    args=training_args,
    train_dataset=train_subset_topic,
    eval_dataset=val_subset_topic,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


C:\Users\SISTERS\AppData\Local\Temp\ipykernel_3480\2952275382.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_source = Trainer(
C:\Users\SISTERS\AppData\Local\Temp\ipykernel_3480\2952275382.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_topic = Trainer(


In [ ]:
train_output_source = trainer_source.train()
print("\n=== Train Output Source ===")
print(train_output_source)

# Topic
train_output_topic = trainer_topic.train()
print("\n=== Train Output Topic ===")
print(train_output_topic)



C:\Users\SISTERS\AppData\Local\Temp\ipykernel_3480\1017584255.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss


#### 5. Evaluación

In [ ]:
# Source
eval_results_source = trainer_source.evaluate()
print("\n=== Validation Results Source ===")
for k, v in eval_results_source.items():
    print(f"{k}: {v:.4f}")

preds_output_source = trainer_source.predict(test_subset_source)
preds_source = np.argmax(preds_output_source.predictions, axis=1)
labels_source = preds_output_source.label_ids
print("\n=== Test Classification Report Source ===")
print(classification_report(labels_source, preds_source, digits=4))

# Topic
eval_results_topic = trainer_topic.evaluate()
print("\n=== Validation Results Topic ===")
for k, v in eval_results_topic.items():
    print(f"{k}: {v:.4f}")

preds_output_topic = trainer_topic.predict(test_subset_topic)
preds_topic = np.argmax(preds_output_topic.predictions, axis=1)
labels_topic = preds_output_topic.label_ids
print("\n=== Test Classification Report Topic ===")
print(classification_report(labels_topic, preds_topic, digits=4))


# ===========================
# GUARDAR MODELOS
# ===========================

trainer_source.save_model("models/distilbert_source")
trainer_topic.save_model("models/distilbert_topic")

tokenizer.save_pretrained("models/distilbert_source")  # el mismo tokenizador sirve para ambos

#### 6. Comparación

## **2. Text Summarization**

Además de la tarea principal de clasificación de sesgo, el dataset de noticias utilizado permite abordar de forma natural otras tareas relevantes de Procesamiento de Lenguaje Natural. Una de las más adecuadas en este contexto es el text summarization, ya que las noticias suelen ser documentos relativamente largos que contienen información redundante o secundaria. La capacidad de generar resúmenes concisos y coherentes resulta especialmente útil en escenarios reales, como la lectura rápida de información, la indexación de contenidos o la visualización eficiente de grandes volúmenes de noticias.

El text summarization consiste en generar una versión más corta de un texto original conservando sus ideas principales y su significado global. En este proyecto se aborda como una tarea abstractive summarization, en la que el modelo no se limita a extraer frases del texto original, sino que es capaz de reformular el contenido y generar nuevas frases que sinteticen la información más relevante. Este enfoque es especialmente adecuado para textos periodísticos, donde la información clave puede estar distribuida a lo largo del documento y no concentrada en frases concretas.

Para llevar a cabo esta tarea se ha seleccionado un modelo basado en Transformers disponible en Hugging Face, concretamente BART (Bidirectional and Auto-Regressive Transformers). BART es una arquitectura encoder–decoder diseñada específicamente para tareas de generación de texto, como el resumen, la traducción o la generación de respuestas. En particular, se utiliza una versión de BART preentrenada para el resumen de noticias, entrenada previamente sobre grandes corpus periodísticos, lo que la hace especialmente adecuada para el tipo de textos presentes en nuestro dataset.

El uso de un modelo preentrenado permite aplicar el resumen sin necesidad de realizar fine-tuning adicional, lo cual es una ventaja importante dadas las limitaciones de recursos computacionales. De este modo, el modelo puede aplicarse directamente sobre un subconjunto de noticias para generar resúmenes y analizar cualitativamente sus resultados. Esta estrategia cumple con el objetivo de la entrega, que es familiarizarse con otras tareas y arquitecturas de NLP, sin requerir un entrenamiento costoso o evaluaciones complejas.

Los objetivos principales de esta parte del trabajo son, en primer lugar, explorar una tarea de NLP diferente a la clasificación, ampliando el alcance del proyecto más allá de la predicción de etiquetas. En segundo lugar, se busca comprender cómo los modelos generativos basados en Transformers procesan y condensan información textual, y qué tipo de salidas producen cuando se aplican a noticias reales. Finalmente, esta tarea permite reflexionar sobre el potencial uso práctico de el text summarization como complemento a la clasificación de sesgo, por ejemplo, para ofrecer resúmenes rápidos de noticias ya categorizadas.

No obstante, también se esperan varias limitaciones e inconvenientes. En primer lugar, al trabajar sin fine-tuning específico sobre nuestro dataset, los resúmenes generados pueden no capturar siempre los matices más relevantes del texto original o pueden omitir detalles importantes relacionados con el sesgo. Además, los modelos de resumización suelen tener restricciones en la longitud máxima de entrada, lo que obliga a truncar textos largos y puede provocar pérdida de información. Por último, el uso de estos modelos en CPU implica tiempos de ejecución relativamente altos, lo que limita la cantidad de ejemplos que pueden procesarse durante los experimentos.

### 1. Carga de Datos

En esta etapa del proyecto se centra en obtener y preparar la información que se utilizará para generar los resúmenes automáticos. Primero, se accede al conjunto de noticias previamente limpiado y procesado, que contiene toda la información textual organizada en columnas. DEspues se selecciona únicamente la columna que contiene el contenido principal de cada noticia, descartando cualquier información que no sea relevante para la tarea de resumen.

Se eliminan los valores vacíos o nulos para asegurar que todos los textos que se procesen tengan contenido real y significativo. Para facilitar las pruebas y la visualización de resultados, se trabaja inicialmente con un subconjunto representativo de los textos, evitando así procesar el conjunto completo en las etapas iniciales. Además, se descartan los textos demasiado cortos, que no aportarían suficiente información para generar resúmenes coherentes.

El objetivo de esta fase es garantizar que el modelo reciba únicamente información útil y consistente, evitando errores o resultados irrelevantes. Se espera que, al final de este bloque, se tenga una lista de noticias limpias y consistentes, cada una con suficiente contenido para que el modelo pueda producir resúmenes coherentes y significativos, y que facilite la comparación posterior entre los textos originales y los resúmenes generados.

In [25]:
import pandas as pd
import torch
import textwrap
from transformers import pipeline


In [3]:
# Cargar dataset
df = pd.read_csv("data/data_clean/train_clean.csv")

# Nos quedamos solo con la columna de texto
texts = df["content_clean"].dropna().tolist()

print(f"Número total de noticias: {len(texts)}")


Número total de noticias: 27978


In [ ]:
texts_subset = texts[:20]  

# para ignorar textos cortos
texts_subset = [t for t in texts_subset if len(t.strip()) > 20] 


### 2. Selección de Modelo

Para esta tarea se ha escogido BART (Bidirectional and Auto-Regressive Transformers) en su versión “large” entrenada para resumir textos, conocido como facebook/bart-large-cnn. Este modelo combina una arquitectura de encoder-decoder con un entrenamiento específico en tareas de resumen de textos largos, lo que le permite identificar la información más importante y generar resúmenes coherentes y comprensibles. BART es especialmente potente para textos de noticias, ya que puede manejar secuencias extensas y capturar las relaciones contextuales entre diferentes partes del texto, asegurando que los resúmenes mantengan la esencia de la información original.

La elección del modelo también se ha hecho considerando la capacidad computacional disponible. Aunque BART large es un modelo pesado y exigente en recursos, la configuración se ajustaa la CPU cuando la GPU no esté disponible. Esto asegura que los resúmenes se generen de manera eficiente sin comprometer la calidad, adaptándose a las limitaciones del entorno de trabajo.

El objetivo de este bloque es garantizar que el modelo seleccionado pueda resumir las noticias de manera precisa y eficiente, extrayendo la información más relevante y produciendo resultados que sean fácilmente comparables con los textos originales. Se espera que, gracias a la elección de BART large, los resúmenes sean coherentes, informativos y representativos del contenido de cada noticia, permitiendo evaluar de manera confiable el rendimiento del sistema de resumen automático.

In [ ]:
# Definimos el nombre del modelo de resumen que vamos a usar
model_name = "facebook/bart-large-cnn"  # BART large entrenado para resúmenes de noticias

# Creamos el pipeline 
summarizer = pipeline(
    "summarization",        
    model=model_name,       
    tokenizer=model_name,      
    device=0 if torch.cuda.is_available() else -1 
)

Device set to use cpu


### 3. Generación de Resumenes

Este bloque constituye el núcleo de la tarea de resumen, ya que es el encargado de procesar cada noticia y producir un resumen coherente y completo. La estrategia central consiste en tomar un texto, analizarlo en profundidad y extraer su información más relevante, asegurando que el resultado sea comprensible y representativo de todo el contenido original.

Dado que los textos pueden ser muy largos, se utiliza un enfoque de procesamiento por fragmentos. Cada noticia se divide en segmentos de tamaño limitado, adaptados a la capacidad del modelo, para evitar que se supere el límite de entrada que los transformadores pueden procesar de una sola vez. Esto permite que incluso los textos extensos puedan ser resumidos sin perder información crítica. Cada fragmento se analiza de manera independiente y se genera un resumen parcial para ese segmento.

Para mejorar la adaptabilidad del modelo, se ajustan los parámetros de longitud máxima del resumen en función del tamaño del fragmento, de manera que los fragmentos muy cortos no generen resúmenes desproporcionados o incompletos. Esto asegura que los resúmenes parciales sean coherentes y mantengan un equilibrio adecuado entre concisión y exhaustividad.

Una vez que se han generado los resúmenes de todos los fragmentos, estos se unen para formar el resumen final de la noticia completa. De esta forma, se preserva la información esencial de cada parte del texto original, garantizando que el resultado global refleje de manera fiel el contenido total.

Durante todo el proceso se contempla la posibilidad de errores o excepciones, como fragmentos que puedan exceder la capacidad del modelo o problemas imprevistos en la generación. En esos casos, se maneja el error de manera controlada, registrando el fragmento problemático y continuando con los demás, lo que asegura la robustez del sistema frente a textos complejos o con irregularidades.

Finalmente, este bloque se aplica iterativamente a cada texto del conjunto seleccionado, generando un resumen individual para cada noticia. El resultado esperado es que cada resumen sea claro, coherente y representativo, facilitando la comparación con el texto original y cumpliendo con el objetivo principal del proyecto: automatizar la síntesis de información relevante de manera eficiente y confiable.

In [ ]:
# Resume un texto dividiéndolo en fragmentos si es demasiado largo para el modelo.

def summarize_text(text, max_length=130, min_length=40, chunk_size=1024):

    summaries = []
    
    # dividir el texto en fragmentos
    for i in range(0, len(text), chunk_size):
        chunk = text[i:i+chunk_size]
        #se ajusta por si es muy corto el texto original
        adjusted_max = min(max_length, len(chunk)//2)
        try:
            summary = summarizer(
                chunk,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )[0]['summary_text']
            summaries.append(summary)
        except Exception as e:
            # por si falla
            summaries.append("")
            print(f"Error en el fragmento {i//chunk_size + 1}: {e}")
    
    # unir los resúmenes parciales
    return " ".join(summaries)


In [ ]:
summaries = []  # Lista donde almacenaremos los resúmenes generados

# Iteramos sobre cada texto del subconjunto de noticias
for i, text in enumerate(texts_subset):
    try:
        # Generamos el resumen del texto usando la función principal
        summary = summarize_text(text)
        summaries.append(summary)
        # mensaje informativo 
        print(f"Resumen {i+1} generado, longitud texto: {len(text)}")
    except Exception as e:
        # En caso de que falle la generación del resumen, añadimos un string vacío
        summaries.append("")
        # mensaje de error 
        print(f"Error en el texto {i+1} (longitud {len(text)}): {e}")


Your max_length is set to 130, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


Resumen 1 generado, longitud texto: 2471
Resumen 2 generado, longitud texto: 4081
Resumen 3 generado, longitud texto: 4807
Resumen 4 generado, longitud texto: 9124
Resumen 5 generado, longitud texto: 4000
Resumen 6 generado, longitud texto: 3933
Resumen 7 generado, longitud texto: 8926


Your max_length is set to 130, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


Resumen 8 generado, longitud texto: 5473


Your max_length is set to 130, but your input_length is only 40. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


...

Resumen 18 generado, longitud texto: 7260


Your max_length is set to 130, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


Resumen 19 generado, longitud texto: 8276


Your max_length is set to 130, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Resumen 20 generado, longitud texto: 10292


### 4. Visualización de Resultados

En este proyecto, hemos conseguido extraer automáticamente la información más relevante de un conjunto de noticias, generando resúmenes coherentes y comprensibles a partir de textos originales extensos. Para ello hemos seleccionado  el modelo BART large, entrenado específicamente para tareas de resumen. Esta elección nos ha permitido equilibrar la calidad de los resúmenes con la capacidad computacional disponible, adaptándonos a la CPU . De esta manera, aseguramos que nuestro sistema pueda funcionar de manera eficiente incluso con limitaciones de hardware, lo cual es fundamental para aplicaciones prácticas.

Uno de los logros más importantes ha sido implementar un método que divide los textos largos en fragmentos manejables para el modelo. Gracias a esta estrategia hemos podido superar la limitación de tamaño de entrada de los transformadores y generar resúmenes parciales que, al unirse, mantienen la coherencia y la información esencial de cada noticia. Este enfoque nos ha permitido procesar textos extensos sin perder contenido crítico, aunque observamos que algunos resúmenes parciales pueden repetir información o generar transiciones no muy buenas entre fragmentos.

Durante el desarrollo del proyecto también identificamos algunas dificultades y limitaciones. Los textos muy cortos o con estructuras poco convencionales a veces produjeron resúmenes incompletos o vacíos, lo que nos enseñó la importancia de filtrar adecuadamente los datos antes de procesarlos. Además, aunque BART large genera resúmenes detallados, en ciertos casos los resultados fueron más extensos de lo esperado, ya que el modelo intenta conservar muchos detalles de la noticia. Esto nos indica que en futuras implementaciones podría ser útil ajustar con mayor precisión los parámetros de longitud máxima y mínima o tambien considerar modelos alternativos según la extensión y el tipo de noticia.

Otro aspecto clave ha sido la visualización y presentación de los resultados. Al mostrar los textos originales y sus resúmenes en párrafos legibles, con un ancho de línea limitado, hemos podido comparar de manera rápida y clara la fidelidad de los resúmenes. Este enfoque nos permitió identificar fácilmente que resúmenes capturan correctamente la información principal y cuales requieren ajustes, proporcionando un feedback inmediato sobre el desempeño del modelo.

Con estos resultados dedujimos varias cosas. Primero, es posible automatizar de manera eficiente la síntesis de noticias largas, reduciendo significativamente el tiempo que dedicaríamos a revisar grandes volúmenes de información. Segundo, la calidad de los resúmenes depende tanto del modelo elegido como de la preparación y filtrado de los datos, lo que demuestra la importancia de contar con un dataset limpio y bien seleccionado. Tercero, aunque la automatización es efectiva, los modelos actuales aún pueden generar repeticiones o incluir detalles irrelevantes, por lo que consideramos recomendable realizar revisiones humanas cuando se busque alta precisión.

In [ ]:

# Creamos un DataFrame que combina los textos originales con los resúmenes generados
df_summaries = pd.DataFrame({
    "Original_Text": texts_subset,
    "Generated_Summary": summaries
})

# Mostramos las primeras filas
df_summaries.head()


,Original_Text,Generated_Summary
0,besides his most recent trip to quetta mr raha...,mr rahami visited karachi pakistan in 2005 an...
1,poll prestigious colleges wo nt make you happi...,Gallup poll finds prestigious colleges don't m...
2,house speaker paul ryan at a private dinner ea...,House speaker paul ryan at a private dinner ea...
3,cnn president donald trump has reason to hope ...,Donald trump has reason to hope his luck is ch...
4,the controversial immigrationreform bill that ...,The controversial immigrationreform bill that ...


In [ ]:
#guardamos resumenes
df_summaries.to_csv("news_summaries_bart.csv", index=False)

In [ ]:
width = 100

#MOstramos ejemplos de lso resumenes hechos
for i in range(3):
    original_text = df_summaries.loc[i, "Original_Text"][:500] + "..."
    generated_summary = df_summaries.loc[i, "Generated_Summary"]

    print("\n--- NOTICIA ORIGINAL ---\n")
    print("\n".join(textwrap.wrap(original_text, width=width)))
    
    print("\n--- RESUMEN GENERADO ---\n")
    print("\n".join(textwrap.wrap(generated_summary, width=width)))
    print("\n" + "="*width + "\n")  


--- NOTICIA ORIGINAL ---

besides his most recent trip to quetta mr rahami visited karachi pakistan in 2005 both of those
cities reputations have become entwined with the militant groups who have sheltered there karachi as
a haven for the pakistani taliban and al qaeda and quetta as the headquarters of the exiled afghan
taliban leadership but both cities are also home to generations of afghans who have fled violence in
their home country much about his new jersey life did seem unremarkable amarjit singh a limousine
drive...

--- RESUMEN GENERADO ---

 mr rahami visited karachi pakistan in 2005 and quetta in 2005 both of those cities reputations have
become entwined with the militant groups who have sheltered there. Both cities are also home to
generations of afghans who have fled violence in their home country much about his new jersey life
did seem unremarkable. Mr rahami had a daughter with a high school girlfriend according to friends
reached at her home on monday night she decline